### The Gears of Neural Networks: Tensor Operations

#### Element-wise operations

In [57]:
import numpy as np

ReLU formula in math form:

$$
\operatorname{ReLU}(x) = \max(0, x) =
\begin{cases}
0, & x < 0 \\
x, & x \ge 0
\end{cases}
$$

In [58]:
def naive_relu(x):
    assert len(x.shape) == 2
    x = x.copy()
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            x[i, j] = max(x[i, j], 0)
    return x

In [59]:
x = np.array([
    [-2.0, -1.0, 0.0],
    [1.0, 2.0, -3.0]
])

naive_relu(x)

array([[0., 0., 0.],
       [1., 2., 0.]])

#### What `naive_relu` is doing

The `naive_relu(x)` function applies the ReLU (Rectified Linear Unit) activation element-wise to a 2D NumPy array `x`. It first makes a copy of `x` so the original isn’t modified, then loops over every element and replaces any negative value with `0`, leaving positive values unchanged. The result is a new array where all entries are `>= 0`.

In [60]:
def naive_add(x, y):
    assert len(x.shape) == 2
    assert x.shape == y.shape
    x = x.copy()
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            x[i, j] += y[i, j]
    return x

In [61]:
x = np.array([[1.0, -2.0]])
y = np.array([[0.5, -2.0]])

naive_add(x, y)

array([[ 1.5, -4. ]])

#### What `naive_add` is doing

The `naive_add(x, y)` function performs element-wise addition on two 2D NumPy arrays of the same shape. It first checks that both inputs are 2D and have identical shapes, then makes a copy of `x` so the original isn’t modified. It loops over every index `(i, j)` and adds `y[i, j]` to `x[i, j]`, returning the resulting summed array.

#### Numpy implementation

In [62]:
z = x + y 
print("Element-wise Addition:", z)

z = np.maximum(z, 0.0)
print("Element-wise ReLU:", z)
# z

Element-wise Addition: [[ 1.5 -4. ]]
Element-wise ReLU: [[1.5 0. ]]


In [76]:
import time

x = np.random.random((20, 100))
y = np.random.random((20, 100))

t0 = time.time()
for _ in range(1000):
    z = x + y
    z = np.maximum(z, 0.0)
print("Took: {0:.3f}s".format(time.time() - t0))

Took: 0.004s


In [75]:
t0 = time.time()
for _ in range(1000):
    z = naive_add(x, y)
    z = naive_relu(z)
print("Took: {0:.3f}s".format(time.time() - t0))

Took: 0.793s
